In [1]:
import tensorflow as tf

import os, pathlib, warnings, urllib, shutil

warnings.filterwarnings("ignore")

In [2]:
if not os.path.exists("Helper.py"):

  giturl = "https://raw.githubusercontent.com/Kelnite/Hut/refs/heads/master/Helper.py"

  urllib.request.urlretrieve(giturl, "Helper.py")

In [3]:
from Helper import GetFile

giturl = "https://github.com/dicodingacademy/assets/releases/download/release/"

dataset = "rockpaperscissors.zip"

GetFile(giturl, dataset, "/content", unpack=True, file_type="zip")

'/content'

In [4]:
root = "/content/rockpaperscissors/rps-cv-images"

os.listdir(root)

['rock', 'paper', 'README_rpc-cv-images.txt', 'scissors']

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

rescale = 1.0 / 255.0

trainset = ImageDataGenerator(
  rescale = rescale,
  rotation_range = 10,
  validation_split = 0.3,
  zoom_range = 0.2,
  shear_range = 0.2,
  fill_mode = 'nearest'
)

validset = ImageDataGenerator(
  rescale=rescale,
  validation_split = 0.3
)

In [ ]:
total_batch = 128

train = trainset.flow_from_directory(
  root,
  target_size = (150, 150),
  class_mode = "categorical",
  batch_size = total_batch,
  subset = "training"
)

In [7]:
labeler = list(train.class_indices.keys())

labeler

['paper', 'rock', 'scissors']

In [ ]:
valid = validset.flow_from_directory(
  root,
  target_size = (150, 150),
  class_mode = "categorical",
  batch_size = total_batch,
  subset = "validation"
)

In [ ]:
from tensorflow.keras.models import Sequential

from tensorflow.keras import layers

model = Sequential([
  layers.Conv2D(16, 3, activation="relu", input_shape=(150, 150, 3)),
  layers.MaxPool2D((2, 2)),
  layers.Conv2D(32, 3, activation="relu"),
  layers.MaxPool2D((2, 2)),
  layers.Conv2D(64, 3, activation="relu"),
  layers.MaxPool2D((2, 2)),
  layers.Flatten(),
  layers.Dropout(0.3),
  layers.Dense(128, activation="relu"),
  layers.Dense(3, activation="softmax")
])

loss = "categorical_crossentropy"

model.compile(optimizer="adam", loss=loss, metrics=["accuracy"])

In [ ]:
hist = model.fit(train, epochs=10, validation_data=valid)